In [ ]:
# 
!pip install -q google.generativeai

In [ ]:
# This script demonstrates how to use the PaLM API with Google Cloud credentials.
# It assumes you have a file named credentials.py with the following content:
import os
import google.generativeai as palm
from google.api_core import client_options
# Get credentials from credentials.py
from credentials import credentials

In [ ]:
# Configure PALM client
palm.configure(
    api_key = credentials["api_key"], # API key
    transport = "rest", # Use REST transport for local development
    client_options = client_options.ClientOptions(api_endpoint=os.getenv("GOOGLE_API_BASE")), # Use local API endpoint for local development

)

In [ ]:
# Explore available models & their generation methods
for model in palm.list_models():
    print("Model",{model.name})
    print(f"description: {model.description}")
    print("Supported Generation Methods:", model.supported_generation_methods)

Model {'models/chat-bison-001'}
description: Chat-optimized generative language model.
Supported Generation Methods: ['generateMessage', 'countMessageTokens']
Model {'models/text-bison-001'}
description: Model targeted for text generation.
Supported Generation Methods: ['generateText', 'countTextTokens', 'createTunedTextModel']
Model {'models/embedding-gecko-001'}
description: Obtain a distributed representation of a text.
Supported Generation Methods: ['embedText', 'countTextTokens']


In [ ]:
# Save models that support text generation
text_generators = [model for model in palm.list_models() if "generateText" in model.supported_generation_methods]

In [ ]:
text_gen_model = text_generators[0] # Use the first model that supports text generation

In [ ]:
text_gen_model 

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='Text Bison',
      description='Model targeted for text generation.',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

In [ ]:
response = palm.chat(messages=["Hello."]) # Generate a response to the message "Hello."
print(response.last) # Print the last message in the response

Hello! How can I help you today?


ChatResponse(model='models/chat-bison-001',
             context='',
             examples=[],
             messages=[...],
             temperature=None,
             candidate_count=None,
             candidates=[...],
             filters=[],
             top_p=None,
             top_k=None,
             _client=<google.ai.generativelanguage_v1beta3.services.discuss_service.client.DiscussServiceClient object at 0x7f68b9688910>)

In [ ]:
response.reply("I need help with django") # Generate a reply to the message "I need help with django."
print(response.last) # Print the last message in the response

ChatResponse(model='models/chat-bison-001',
             context='',
             examples=[],
             messages=[...],
             temperature=None,
             candidate_count=None,
             candidates=[...],
             filters=[],
             top_p=None,
             top_k=None,
             _client=<google.ai.generativelanguage_v1beta3.services.discuss_service.client.DiscussServiceClient object at 0x7f68b9688910>)

In [ ]:
from google.api_core import retry

In [ ]:
@retry.Retry(deadline=60, predicate=retry.if_exception_type(Exception)) # Retry the function if it raises an exception
def generate_text(prompt, model = text_gen_model,temperature = 0.1): # Generate text using the specified model and prompt
    return palm.generate_text(prompt=prompt, model=model, temperature=temperature)

In [ ]:
prompt = """I have a django project where I make users upload images and then I process them using this view but I want an other view that makes the processing for a displayed image selected by a user: def upload_image(request):
    \"\"\"
    Handles image upload, processing, and classification.

    Inputs:
    - request (HttpRequest): The HTTP request object containing the uploaded image.

    Outputs:
    - JsonResponse: A JSON response containing prediction results or an error message.
    \"\"\"

    if request.method == 'POST':
        form = UploadImageForm(request.POST, request.FILES)
        if form.is_valid():

            image_instance = form.save(commit=False)

            image = Image.open(image_instance.image)

            # Convert the image to grayscale
            grayscale = convert_to_grayscale(image)

            # Preprocess the image for the model
            preprocessed_image = preprocess_image(grayscale, target_size=input_shape)

            # Expand dimensions to match model input
            preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
            print('predicting...')

            # Perform the image classification and get a prediction
            prediction = loaded_model.predict(preprocessed_image)
            print('prediction done')
            # Determine the predicted label based on the prediction score
            if prediction <= 0.3:
                predicted_label = 'Defected Condition'
            elif prediction > 0.3:
                predicted_label = 'Non Defected Condition'
            else:
                predicted_label = 'unknown, there must be an error'
            # Convert the original image to bytes
            original_image_data = image.tobytes()
            # Create a unique blob name based on the prediction label
            timestamp = timezone.now().strftime("%Y%m%d%H%M%S")
            blob_name = f"{timestamp}_{predicted_label}"

            # Upload the image to Azure Blob Storage
            upload_result = upload_image_to_blob(original_image_data, blob_name)
            print('upload done')
            if upload_result:

                print('returning response')
                # Image was successfully uploaded to Azure Blob Storage
                # Save the image instance with the blob_name
                image_instance.blob_name = blob_name
                image_instance.save()
                response_data = {
                    'prediction': predicted_label,
                    'prediction_probabilityOK': prediction.tolist()[0][0],
                    'prediction_probabilityDefect': 1- prediction.tolist()[0][0],
                }
                return JsonResponse(response_data)
            else:
                # Handle the case where the image upload to Blob Storage failed
                response_data = {
                    'error': 'Failed to upload image to Azure Blob Storage',
                }
                return JsonResponse(response_data, status=500)  # Return a 500 Internal Server Error status

    else:
        form = UploadImageForm()
    return render(request, 'ClassifierModel/upload_image.html', {'form': form})
"""

In [ ]:
completion = generate_text(prompt) # Generate text using the PaLM API

In [ ]:
print(completion.result) # Print the generated text

```python
def process_image(request, image_id):
    """
    Processes an image that has been selected by a user.

    Inputs:
    - request (HttpRequest): The HTTP request object.
    - image_id (str): The ID of the image to be processed.

    Outputs:
    - JsonResponse: A JSON response containing prediction results or an error message.
    """

    # Get the image instance from the database
    image_instance = Image.objects.get(id=image_id)

    # Get the image data from Azure Blob Storage
    image_data = get_image_from_blob(image_instance.blob_name)

    # Convert the image to grayscale
    grayscale = convert_to_grayscale(image_data)

    # Preprocess the image for the model
    preprocessed_image = preprocess_image(grayscale, target_size=input_shape)

    # Expand dimensions to match model input
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

    # Perform the image classification and get a prediction
    prediction = loaded_model.predict(preprocessed_image)

In [ ]:
prompt = "Show me how to use docker for my machine learning projects" # Set a prompt for the PaLM API

In [ ]:
completion = generate_text(prompt) # Generate text using the PaLM API

In [ ]:
# Set a prompt for the PaLM API
prompt = "Based on the Idea that: 'In machine learning,there is a model centric approach and a data centric approach' generate 10 interview questions."

In [ ]:
# Generate text using the PaLM API
completion = generate_text(prompt)

In [ ]:
print(completion.result) # View the generated text

1. What is the difference between a model-centric approach and a data-centric approach to machine learning?
2. What are the advantages and disadvantages of each approach?
3. When is it better to use a model-centric approach?
4. When is it better to use a data-centric approach?
5. How can you decide which approach to use for a particular problem?
6. What are some of the challenges of using a model-centric approach?
7. What are some of the challenges of using a data-centric approach?
8. How can you mitigate the challenges of each approach?
9. What are the latest research advances in model-centric and data-centric machine learning?
10. What are the future trends in model-centric and data-centric machine learning?


In [ ]:
# Save the generated text to a file
with open("interview_questions.txt", "w") as f:
    f.write(completion.result) # Write the generated text to the file

In [ ]:
# Save models that support text generation
chat_generators =    [model for model in palm.list_models() if "generateMessage" in model.supported_generation_methods] # 

In [ ]:
chat_gen_model =chat_generators[0] 

In [ ]:
# Set prompt template
prompt_template = """
{priming}

{question}

{decorator}

solution:
"""


In [ ]:
# Set prompt variables
priming_text = "You are an expert at writing clear, concise, Python code."
question = "create a doubly linked list"
decorator = "The code should be well documented and include type hints."

In [ ]:
# Format the prompt using the template and variables
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)